In [ ]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, BatchNormalization,Input
from tensorflow.keras.optimizers import SGD
import numpy as np
from tensorflow.keras.regularizers import l2

In [ ]:
class Cifar10:
    def __init__(self, train=True):
        self.num_classes = 10
        self.weight_decay = 0.0005
        self.x_shape = [32,32,3]

        self.model = self.build_model()
        if train:
            self.model = self.train(self.model)
        else:
            self.model.load_weights('cifar10vgg.h5')

    def build_model(self):
        model = Sequential()
        # First block
        model.add(Conv2D(64, (3, 3), padding='same', kernel_regularizer=l2(self.weight_decay), input_shape=self.x_shape))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.3))
        model.add(Conv2D(64, (3, 3), padding='same', kernel_regularizer=l2(self.weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(MaxPooling2D(pool_size=(2, 2)))

        # Second block
        model.add(Conv2D(128, (3, 3), padding='same', kernel_regularizer=l2(self.weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Conv2D(128, (3, 3), padding='same', kernel_regularizer=l2(self.weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(MaxPooling2D(pool_size=(2, 2)))

        # Third block
        model.add(Conv2D(256, (3, 3), padding='same', kernel_regularizer=l2(self.weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Conv2D(256, (3, 3), padding='same', kernel_regularizer=l2(self.weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(MaxPooling2D(pool_size=(2, 2)))

        # Fourth block
        model.add(Conv2D(512, (3, 3), padding='same', kernel_regularizer=l2(self.weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Conv2D(512, (3, 3), padding='same', kernel_regularizer=l2(self.weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(MaxPooling2D(pool_size=(2, 2)))

        # Fifth block
        model.add(Conv2D(512, (3, 3), padding='same', kernel_regularizer=l2(self.weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Conv2D(512, (3, 3), padding='same', kernel_regularizer=l2(self.weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.5))

        # Final dense layers
        model.add(Flatten())
        model.add(Dense(512, kernel_regularizer=l2(self.weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.5))
        model.add(Dense(self.num_classes, activation='softmax'))

        return model

    def normalize(self, X_train, X_test):
        mean = np.mean(X_train, axis=(0, 1, 2, 3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train - mean) / (std + 1e-7)
        X_test = (X_test - mean) / (std + 1e-7)
        return X_train, X_test


    def save(self):
        model_json = model.to_json()
        with open('CIFAR10_model.json', 'w') as json_file:
            json_file.write(model_json)
        model.save_weights('CIFAR10_model.weights.h5')

        model.save('CIFAR_10.hdf5')


    def train(self, model):
        batch_size = 128
        maxepochs = 250
        learning_rate = 0.1
        lr_drop = 20

        (x_train, y_train), (x_test, y_test) = cifar10.load_data()
        x_train, x_test = self.normalize(x_train, x_test)
        y_train = tf.keras.utils.to_categorical(y_train, self.num_classes)
        y_test = tf.keras.utils.to_categorical(y_test, self.num_classes)

        def lr_scheduler(epoch):
            return learning_rate * (0.5 ** (epoch // lr_drop))
        reduce_lr = tf.keras.callbacks.LearningRateScheduler(lr_scheduler)

        datagen = ImageDataGenerator(
          featurewise_center=False,
          samplewise_center=False,
          featurewise_std_normalization=False,
          samplewise_std_normalization=False,
          zca_whitening=False,
          rotation_range=15,
          width_shift_range=0.1,
          height_shift_range=0.1,
          horizontal_flip=True,
          vertical_flip=False)
        datagen.fit(x_train)

                # EarlyStopping callback
        early_stopping = EarlyStopping(
            monitor='val_loss',  # Monitor validation loss
            min_delta=0.001,     # Minimum change to qualify as an improvement
            patience=15,         # Stop after 13 epochs with no improvement
            verbose=1,           # Print messages
            restore_best_weights=True  # Restore model weights from the epoch with the best value of the monitored quantity.
        )

        sgd = SGD(learning_rate=learning_rate, momentum=0.9, nesterov=True)
        model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])


        model.fit(datagen.flow(x_train, y_train, batch_size=batch_size),
              epochs=maxepochs, validation_data=(x_test, y_test),
              callbacks=[early_stopping, reduce_lr], verbose=1)

        test_loss, test_accuracy = model.evaluate(x_test, y_test, verbose=1)
        print(f'Test Accuracy: {test_accuracy*100:.2f}%')
        model.summary()

        return model

if __name__ == '__main__':
    model = Cifar10(train=True)
    (x_train, y_train), (x_test, y_test) = cifar10.load_data()
    x_test = x_test.astype('float32')
    x_test, _ = model.normalize(x_test, x_test)  # Normalize test data
    y_test = tf.keras.utils.to_categorical(y_test, model.num_classes)

170498071/170498071 [==============================] - 5s 0us/step
Epoch 1/250
391/391 [==============================] - 55s 109ms/step - loss: 8.7239 - accuracy: 0.2659 - val_loss: 6.3132 - val_accuracy: 0.2946 - lr: 0.1000
Epoch 2/250
391/391 [==============================] - 39s 98ms/step - loss: 4.6757 - accuracy: 0.4366 - val_loss: 3.6324 - val_accuracy: 0.4597 - lr: 0.1000
Epoch 3/250
391/391 [==============================] - 39s 100ms/step - loss: 2.8432 - accuracy: 0.5475 - val_loss: 2.4193 - val_accuracy: 0.5478 - lr: 0.1000
Epoch 4/250
391/391 [==============================] - 38s 97ms/step - loss: 1.9525 - accuracy: 0.6266 - val_loss: 1.7899 - val_accuracy: 0.6154 - lr: 0.1000
Epoch 5/250
391/391 [==============================] - 39s 98ms/step - loss: 1.5236 - accuracy: 0.6834 - val_loss: 1.4810 - val_accuracy: 0.6843 - lr: 0.1000
Epoch 6/250
391/391 [==============================] - 38s 98ms/step - loss: 1.3690 - accuracy: 0.7154 - val_loss: 1.4162 - val_accuracy: 0.6

In [ ]:
model.model.save('CIFAR_10.hdf5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
